In [219]:
# Import libraries
from bs4 import BeautifulSoup
import csv
import json
import pandas as pd
import numpy as np
import re
import requests
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
import time
import warnings
import random
import csv
warnings.filterwarnings("ignore")

In [220]:
# Import csv file
cities_path = "../data/hotels_items.csv"

In [221]:
# Create dataframe
cities_df = pd.read_csv(cities_path, usecols = ["city"])

In [222]:
cities_df.drop_duplicates(inplace = True)

In [223]:
city_list = cities_df["city"].tolist()

In [224]:
# # Create empty columns for the values that will be pulled through API
hotels_df = pd.DataFrame()
hotels_df["rating"] = np.nan
hotels_df["name"] = None
hotels_df["address"] = None
hotels_df["locality"] = None
hotels_df["landmarks"] = None
hotels_df["price"] = np.nan

In [225]:
# List lenght
len(city_list)

27194

In [226]:
options = Options()
options.headless = True
#options.add_argument("--window-size=1024x768")
driver = webdriver.Chrome('/usr/bin/chromedriver', chrome_options=options)
url = "https://www.hotels.com/"

#driver.implicitly_wait(2)

with open(r'../data/hotels_com_scrape.csv', 'a') as f:
    csv_writer = csv.writer(f)
    for city in list(city_list):
        #print('Processing: ' + city)

        searchKey = city
        checkInDate = '08/27/2020' #Format %m/%d/%Y
        checkOutDate = '08/29/2020' #Format %m/%d/%Y

        driver.get(url)

        searchKeyElement = driver.find_elements_by_name("q-destination")
        checkInElement = driver.find_elements_by_name("q-localised-check-in")
        checkOutElement = driver.find_elements_by_name("q-localised-check-out")
        submitButton = driver.find_elements_by_css_selector("button.cta-strong")
        closeIconForPopup = driver.find_elements_by_css_selector("span.close-icon")
        closeTypeahead = driver.find_elements_by_css_selector("button.cta-link")
        closeTypeahead = driver.find_elements_by_css_selector("button.cta-link")

        if searchKeyElement and checkInElement and checkOutElement and closeIconForPopup:
            #print('All search elements found.')
            searchKeyElement[0].clear()
            searchKeyElement[0].send_keys(searchKey)
            checkInElement[0].clear()
            checkInElement[0].send_keys(checkInDate)
            checkOutElement[0].clear()
            checkOutElement[0].send_keys(checkOutDate)
            closeIconForPopup[0].click()
            time.sleep(1)
            
            try:
                submitButton[0].click()
            except:                
                # Try to close the the autocomplete box
                time.sleep(1)
                searchKeyElement[0].send_keys(Keys.ARROW_DOWN)
                time.sleep(1)
                searchKeyElement[0].send_keys(Keys.ENTER)
            
            time.sleep(1)
            #print('Pressed the submit button.')

            #print('Scrolling a few times.')
            body_element = driver.find_element_by_tag_name('body')
            
            for i in range(5):
                body_element.send_keys(Keys.END)
            
            time.sleep(1)
            #print(driver.page_source)
            hotels = driver.find_elements_by_css_selector('section.hotel-wrap')

            for index, hotel in enumerate(hotels):
                hotel_name = hotel.find_element_by_css_selector('a.property-name-link').text.strip()
                try:
                    hotel_rating = float(hotel.find_element_by_css_selector('div.ta-logo').text.split()[-1])
                except:
                    hotel_rating = "na"
                hotel_address = hotel.find_element_by_css_selector('span.address').text
                hotel_locality = hotel.find_element_by_css_selector("a.map-link").text.strip()
                hotel_landmarks = hotel.find_element_by_css_selector("ul.property-landmarks").text
                try:
                    hotel_price = float(hotel.find_element_by_css_selector("a.price-link").find_element_by_tag_name("ins").text.strip("$"))
                except:
                    hotel_price = "na"

                #print("Saving...")
                row = [city, hotel_name, hotel_rating, hotel_address, hotel_locality, hotel_landmarks, hotel_price]
                print(row)
                csv_writer.writerow(row)
        #print("Done!")
        city_list.remove(city)
        print("\n" + str(len(city_list)) + " cities remaining. \n")

driver.quit()
print("We done.")

['Mutterstadt, Germany', 'RheinCity Hotel', 3.5, 'Zollhofstraße 11, Ludwigshafen, RP, 67059, Germany', 'Ludwigshafen', '5.4 miles to Mutterstadt\n3.3 miles to Mannheim (MHG)', 107.0]
['Mutterstadt, Germany', 'NYX Hotel Mannheim by Leonardo Hotels', 4.0, 'F4, 4-11, Mannheim, BW, 68159, Germany', 'Mannheim', '6.2 miles to Mutterstadt\n2.8 miles to Mannheim (MHG)', 78.0]
['Mutterstadt, Germany', 'Radisson Blu Hotel, Mannheim', 4.5, 'Quadrant Q7, 27, Mannheim, 68161, Germany', 'Mannheim', '6.4 miles to Mutterstadt\n2.3 miles to Mannheim (MHG)', 'na']
['Mutterstadt, Germany', 'ACHAT Comfort Frankenthal/Pfalz', 3.5, 'Mahlastrasse 18, Frankenthal, RP, 67227, Germany', 'Frankenthal', '5.9 miles to Mutterstadt\n8.3 miles to Mannheim (MHG)', 84.0]
['Mutterstadt, Germany', 'Leonardo Royal Hotel Mannheim', 3.5, 'Augustaanlage 4-8, Mannheim, BW, 68165, Germany', 'Mannheim', '6.5 miles to Mutterstadt\n2.0 miles to Mannheim (MHG)', 80.0]
['Mutterstadt, Germany', 'Leonardo Hotel Mannheim City Center',

['Puerto de la Cruz, Spain', 'Sol Costa Atlantis Tenerife', 4.0, 'Avenida Venezuela, 15, Puerto de la Cruz, Tenerife, 38400, Spain', 'Puerto de la Cruz', '0.4 miles to City center\n13 miles to Santa Cruz de Tenerife (TFN-Tenerife North)', 85.0]
['Puerto de la Cruz, Spain', 'RF San Borondon', 4.0, 'Agustín Espinosa, 2, Puerto de la Cruz, Tenerife, 38400, Spain', 'Puerto de la Cruz', '0.4 miles to City center\n14 miles to Santa Cruz de Tenerife (TFN-Tenerife North)', 'na']
['Puerto de la Cruz, Spain', 'Hotel Botanico & The Oriental Spa Garden', 4.5, 'Avenida Richard Yeoward, 1, Puerto de la Cruz, Tenerife, 38400, Spain', 'Puerto de la Cruz', '0.7 miles to City center\n13 miles to Santa Cruz de Tenerife (TFN-Tenerife North)', 'na']
['Puerto de la Cruz, Spain', 'Apartamentos Casablanca', 4.5, 'Calzada de Martianez, 4, Puerto de la Cruz, Tenerife, 38400, Spain', 'Puerto de la Cruz', '0.4 miles to City center\n13 miles to Santa Cruz de Tenerife (TFN-Tenerife North)', 'na']
['Puerto de la Cru


27176 cities remaining. 

['Arosa, Switzerland', 'Hotel Nolda', 4.5, 'Via Crasta 3, St. Moritz, GR, 7500, Switzerland', 'St. Moritz', '22 miles to Arosa\n22 miles to Obersee Park', 'na']
['Arosa, Switzerland', 'Hotel Oberland', 4.5, 'Bergstrasse 25, Triesenberg, 9497, Liechtenstein', 'Triesenberg, Liechtenstein', '24 miles to Arosa\n24 miles to Obersee Park', 123.0]
['Arosa, Switzerland', 'Central Hotel Post', 4.0, 'Poststrasse 11, Chur, 7000, Switzerland', 'Chur', '8.4 miles to Arosa\n8.4 miles to Obersee Park', 'na']
['Arosa, Switzerland', 'Hotel Steffani', 4.0, 'Am Sonnenplatz, St. Moritz, 7500, Switzerland', 'St. Moritz', '21 miles to Arosa\n21 miles to Obersee Park', 206.0]
['Arosa, Switzerland', 'Hotel Waldhaus', 5.0, 'Via da Fex 3, Sils im Engadin-Segl, GR, 7514, Switzerland', 'Sils im Engadin-Segl', '25 miles to Arosa\n25 miles to Obersee Park', 410.0]
['Arosa, Switzerland', 'arthausHOTEL', 4.5, 'Platzstrasse 5, Davos, 7270, Switzerland', 'Davos', '6.7 miles to Arosa\n6.6 mile

['Allensbach, Germany', 'Hotel Kreuzlingen am Hafen', 4.5, 'Seestrasse 50, Kreuzlingen, TG, 8280, Switzerland', 'Kreuzlingen, Switzerland', '7.7 miles to Allensbach\n16 miles to Friedrichshafen (FDH-Friedrichshafen - Lake Constance)', 159.0]
['Allensbach, Germany', 'Hotel Halm Konstanz', 4.0, 'Bahnhofplatz 6, Konstanz, BW, 78462, Germany', 'Konstanz', '7.2 miles to Allensbach\n16 miles to Friedrichshafen (FDH-Friedrichshafen - Lake Constance)', 'na']
['Allensbach, Germany', 'Hotel-Gasthof Storchen', 4.0, 'Aachstraße 17, Uhldingen-Muehlhofen, BW, 88690, Germany', 'Uhldingen-Muehlhofen', '7.8 miles to Allensbach\n14 miles to Friedrichshafen (FDH-Friedrichshafen - Lake Constance)', 142.0]

27170 cities remaining. 

['Rome, Italy', 'Reginella Suites', 4.5, 'Via della Reginella 2, Rome, 186, Italy', 'Navona', '0.7 miles to City center\n14 miles to Rome (FCO-Fiumicino - Leonardo da Vinci Intl.)', 91.0]
['Rome, Italy', 'Relais Donna Lucrezia', 5.0, 'Via Emilia 88, Rome, 00188, Italy', 'Rome C

['New Milton, United Kingdom', 'Willow Barn Hotel & Restaurant', 4.5, 'Gore Rd, New Milton, England, BH25 6SJ, United Kingdom', 'New Milton', '1.5 miles to City center\n0.9 miles to Barton-on-Sea', 71.0]
['New Milton, United Kingdom', 'Rowan Cottage', 'na', '10 Park Road Barton on Sea, New Milton, England, BH25 6QE, United Kingdom', 'New Milton', '1.8 miles to City center\n0.5 miles to Barton-on-Sea', 'na']
['New Milton, United Kingdom', 'Pebble Beach', 4.5, 'Marine Drive, Barton on Sea, New Milton, England, BH25 7DZ, United Kingdom', 'New Milton', '2.5 miles to City center\n0.3 miles to Barton-on-Sea', 'na']

27164 cities remaining. 


27163 cities remaining. 


27162 cities remaining. 

['Catania, Italy', 'Hotel Nettuno', 3.5, 'Viale Ruggero di Lauria 121, Catania, CT, 95126, Italy', 'Picanello-Ognina-Barriera-Canalicchio', '16 miles to City center\n4.5 miles to Catania (CTA-Fontanarossa)', 119.0]
['Catania, Italy', 'Il Leone Blu - Bed & Breakfast', 5.0, 'Via Castello Ursino, 38, Cat

...


1518 cities remaining. 


1517 cities remaining. 


1516 cities remaining. 


1515 cities remaining. 


1514 cities remaining. 


1513 cities remaining. 


1512 cities remaining. 


1511 cities remaining. 


1510 cities remaining. 


1509 cities remaining. 


1508 cities remaining. 


1507 cities remaining. 


1506 cities remaining. 


1505 cities remaining. 


1504 cities remaining. 


1503 cities remaining. 


1502 cities remaining. 


1501 cities remaining. 


1500 cities remaining. 


1499 cities remaining. 


1498 cities remaining. 


1497 cities remaining. 


1496 cities remaining. 


1495 cities remaining. 


1494 cities remaining. 


1493 cities remaining. 


1492 cities remaining. 


1491 cities remaining. 


1490 cities remaining. 


1489 cities remaining. 


1488 cities remaining. 


1487 cities remaining. 


1486 cities remaining. 


1485 cities remaining. 


1484 cities remaining. 


1483 cities remaining. 


1482 cities remaining. 


1481 cities remaining. 


1480 cities


1201 cities remaining. 


1200 cities remaining. 


1199 cities remaining. 


1198 cities remaining. 


1197 cities remaining. 


1196 cities remaining. 


1195 cities remaining. 


1194 cities remaining. 


1193 cities remaining. 


1192 cities remaining. 


1191 cities remaining. 


1190 cities remaining. 


1189 cities remaining. 


1188 cities remaining. 


1187 cities remaining. 


1186 cities remaining. 


1185 cities remaining. 


1184 cities remaining. 


1183 cities remaining. 


1182 cities remaining. 


1181 cities remaining. 


1180 cities remaining. 


1179 cities remaining. 


1178 cities remaining. 


1177 cities remaining. 


1176 cities remaining. 


1175 cities remaining. 


1174 cities remaining. 


1173 cities remaining. 


1172 cities remaining. 


1171 cities remaining. 


1170 cities remaining. 


1169 cities remaining. 


1168 cities remaining. 


1167 cities remaining. 


1166 cities remaining. 


1165 cities remaining. 


1164 cities remaining. 


1163 cities


881 cities remaining. 


880 cities remaining. 


879 cities remaining. 


878 cities remaining. 


877 cities remaining. 


876 cities remaining. 


875 cities remaining. 


874 cities remaining. 


873 cities remaining. 


872 cities remaining. 


871 cities remaining. 


870 cities remaining. 


869 cities remaining. 


868 cities remaining. 


867 cities remaining. 


866 cities remaining. 


865 cities remaining. 


864 cities remaining. 


863 cities remaining. 


862 cities remaining. 


861 cities remaining. 


860 cities remaining. 


859 cities remaining. 


858 cities remaining. 


857 cities remaining. 


856 cities remaining. 


855 cities remaining. 


854 cities remaining. 


853 cities remaining. 


852 cities remaining. 


851 cities remaining. 


850 cities remaining. 


849 cities remaining. 


848 cities remaining. 


847 cities remaining. 


846 cities remaining. 


845 cities remaining. 


844 cities remaining. 


843 cities remaining. 


842 cities remaining. 




553 cities remaining. 


552 cities remaining. 


551 cities remaining. 


550 cities remaining. 


549 cities remaining. 


548 cities remaining. 


547 cities remaining. 


546 cities remaining. 


545 cities remaining. 


544 cities remaining. 


543 cities remaining. 


542 cities remaining. 


541 cities remaining. 


540 cities remaining. 


539 cities remaining. 


538 cities remaining. 


537 cities remaining. 


536 cities remaining. 


535 cities remaining. 


534 cities remaining. 


533 cities remaining. 


532 cities remaining. 


531 cities remaining. 


530 cities remaining. 


529 cities remaining. 


528 cities remaining. 


527 cities remaining. 


526 cities remaining. 


525 cities remaining. 


524 cities remaining. 


523 cities remaining. 


522 cities remaining. 


521 cities remaining. 


520 cities remaining. 


519 cities remaining. 


518 cities remaining. 


517 cities remaining. 


516 cities remaining. 


515 cities remaining. 


514 cities remaining. 




225 cities remaining. 


224 cities remaining. 


223 cities remaining. 


222 cities remaining. 


221 cities remaining. 


220 cities remaining. 


219 cities remaining. 


218 cities remaining. 


217 cities remaining. 


216 cities remaining. 


215 cities remaining. 


214 cities remaining. 


213 cities remaining. 


212 cities remaining. 


211 cities remaining. 


210 cities remaining. 


209 cities remaining. 


208 cities remaining. 


207 cities remaining. 


206 cities remaining. 


205 cities remaining. 


204 cities remaining. 


203 cities remaining. 


202 cities remaining. 


201 cities remaining. 


200 cities remaining. 


199 cities remaining. 


198 cities remaining. 


197 cities remaining. 


196 cities remaining. 


195 cities remaining. 


194 cities remaining. 


193 cities remaining. 


192 cities remaining. 


191 cities remaining. 


190 cities remaining. 


189 cities remaining. 


188 cities remaining. 


187 cities remaining. 


186 cities remaining. 



In [ ]:
driver.quit()